In [1]:
import sys
print(sys.executable)

e:\causal-churn-analysis\.venv\Scripts\python.exe


In [28]:
import sys
!{sys.executable} -m pip install scikit-learn


     --------------------------------------- 10.7/10.7 MB 38.4 MB/s eta 0:00:00
     --------------------------------------- 38.6/38.6 MB 31.2 MB/s eta 0:00:00
     ---------------------------------------- 307.7/307.7 kB ? eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
import joblib

In [2]:
import pandas as pd  
import os
from IPython.display import display 

In [3]:
file_path = os.path.join("..", "data", "WA_Fn-UseC_-Telco-Customer-Churn.csv") 
df = pd.read_csv(file_path)
display(df.head())


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:

df.info()
df.isnull().sum()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

### Convert TotalCharges to Numeric

TotalCharges was read as object. We convert it to numeric, coercing invalid entries to NaN.


In [5]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df = df.dropna(subset=['TotalCharges'])


### Standardize Column Names

Column names are cleaned by:
- Stripping leading/trailing whitespace
- Converting all letters to lowercase
- Replacing spaces with underscores

This standardization ensures consistency and makes column access easier in code (e.g., df.monthlycharges instead of df['Monthly Charges']).


In [ ]:
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")


### Check and Remove Duplicates

We check for and remove:
- Entirely duplicated rows using `df.duplicated()`
- Duplicate customer IDs using `df['customerid'].duplicated()`

No duplicates were found in this dataset, so no cleaning was needed for this step.


In [7]:
df.duplicated().sum()
df['customerid'].duplicated().sum()


np.int64(0)

## Categorical Variable Summary

This cell inspects all object-type (categorical) columns to:

- Count the number of unique values per column
- Display the frequency of each value

This helps identify:
- Categorical columns with potential for encoding
- Columns with high cardinality (e.g., customerID)
- Possible data quality issues (e.g., inconsistent labels)

Observation:
- `customerID` has 7032 unique values (likely an identifier, drop before modeling)
- `gender` and `partner` are clean binary categorical variables


In [8]:
for col in df.select_dtypes(include='object'):
    print(col, "→", df[col].nunique(), "unique values")
    print(df[col].value_counts(), "\n")


customerid → 7032 unique values
customerid
7590-VHVEG    1
5575-GNVDE    1
3668-QPYBK    1
7795-CFOCW    1
9237-HQITU    1
             ..
6840-RESVB    1
2234-XADUH    1
4801-JZAZL    1
8361-LTMKD    1
3186-AJIEK    1
Name: count, Length: 7032, dtype: int64 

gender → 2 unique values
gender
Male      3549
Female    3483
Name: count, dtype: int64 

partner → 2 unique values
partner
No     3639
Yes    3393
Name: count, dtype: int64 

dependents → 2 unique values
dependents
No     4933
Yes    2099
Name: count, dtype: int64 

phoneservice → 2 unique values
phoneservice
Yes    6352
No      680
Name: count, dtype: int64 

multiplelines → 3 unique values
multiplelines
No                  3385
Yes                 2967
No phone service     680
Name: count, dtype: int64 

internetservice → 3 unique values
internetservice
Fiber optic    3096
DSL            2416
No             1520
Name: count, dtype: int64 

onlinesecurity → 3 unique values
onlinesecurity
No                     3497
Yes         

In [ ]:
df['multiplelines'] = df['multiplelines'].replace('No phone service', 'No')
display(df[['multiplelines', 'phoneservice']].head(10))




,multiplelines,phoneservice
0,No,No
1,No,Yes
2,No,Yes
3,No,No
4,No,Yes
5,Yes,Yes
6,Yes,Yes
7,No,No
8,Yes,Yes
9,No,Yes


### Standardize 'multiplelines' Values

The value `'No phone service'` in the `multiplelines` column is equivalent to `'No'` and was replaced to simplify category levels.

#### Check: Confirm the value was removed and replaced


In [16]:
# Check that the replacement worked
print("Unique values in 'multiplelines':")
print(df['multiplelines'].unique())

# Optional assert for safety (will raise an error if replacement failed)
assert 'No phone service' not in df['multiplelines'].unique()


Unique values in 'multiplelines':
['No' 'Yes']


## Standardize Internet-Related Service Columns

The columns below included the category `'No internet service'`, which indicates the customer does not have internet. These values were replaced with `'No'` to simplify the categories and ensure consistent binary responses:

- onlinesecurity  
- onlinebackup  
- deviceprotection  
- techsupport  
- streamingtv  
- streamingmovies

In [ ]:
internet_cols = ['onlinesecurity', 'onlinebackup', 'deviceprotection', 
                 'techsupport', 'streamingtv', 'streamingmovies']

for col in internet_cols:
    df[col] = df[col].replace('No internet service', 'No')


### Check: Verify 'No internet service' was removed from internet-related columns

In [ ]:
for col in internet_cols:
    print(col, "→", df[col].unique())


onlinesecurity → ['No' 'Yes']
onlinebackup → ['Yes' 'No']
deviceprotection → ['No' 'Yes']
techsupport → ['No' 'Yes']
streamingtv → ['No' 'Yes']
streamingmovies → ['No' 'Yes']


### Convert Binary Columns to Numeric (0/1)

The following columns had `'Yes'` and `'No'` values, which were converted to 1 and 0 for modeling purposes:

- partner, dependents, phoneservice, multiplelines
- onlinesecurity, onlinebackup, deviceprotection, techsupport
- streamingtv, streamingmovies, paperlessbilling, churn


In [17]:
binary_cols = [
    'partner', 'dependents', 'phoneservice', 'multiplelines',
    'onlinesecurity', 'onlinebackup', 'deviceprotection',
    'techsupport', 'streamingtv', 'streamingmovies',
    'paperlessbilling', 'churn'
]

df[binary_cols] = df[binary_cols].apply(lambda col: col.map({'Yes': 1, 'No': 0}))


## Check: Validate Binary Columns Contain Only 0 and 1



In [24]:
for col in binary_cols:
    print(f"{col}: {df[col].unique()}")
    assert set(df[col].unique()).issubset({0, 1}), f"{col} contains unexpected values"


partner: [1 0]
dependents: [0 1]
phoneservice: [0 1]
multiplelines: [0 1]
onlinesecurity: [0 1]
onlinebackup: [1 0]
deviceprotection: [0 1]
techsupport: [0 1]
streamingtv: [0 1]
streamingmovies: [0 1]
paperlessbilling: [1 0]
churn: [0 1]


## Label Encoding Mapping for 'Contract'

- 0 = Month-to-month  
- 1 = One year  
- 2 = Two year  

Mapping dictionary (for reference and reuse):  
`{'Month-to-month': 0, 'One year': 1, 'Two year': 2}`


In [30]:

from sklearn.preprocessing import LabelEncoder

le_contract = LabelEncoder()
df['contract_encoded'] = le_contract.fit_transform(df['contract'])

# Save mapping
contract_label_map = dict(zip(le_contract.classes_, le_contract.transform(le_contract.classes_)))


## Check: Verify 'Contract' Label Encoding and Mapping

In [31]:
# Print encoded values to verify
print(df[['contract', 'contract_encoded']].drop_duplicates().sort_values('contract_encoded'))

# Optional reverse map for later use
contract_reverse_map = {v: k for k, v in contract_label_map.items()}


          contract  contract_encoded
0   Month-to-month                 0
1         One year                 1
11        Two year                 2


In [33]:
joblib.dump(le_contract, 'contract_label_encoder.pkl')


['contract_label_encoder.pkl']

### Label Encoding: Binary Categorical Columns

These columns contain only two unique text values (e.g., 'Yes'/'No'), so we use LabelEncoder to convert them to 0/1 for modeling.


In [34]:
binary_cols = ['gender', 'partner', 'dependents', 'phoneservice', 'paperlessbilling']
label_encoders = {}

for col in binary_cols:
    le = LabelEncoder()
    df[f'{col}_encoded'] = le.fit_transform(df[col])
    label_encoders[col] = dict(zip(le.classes_, le.transform(le.classes_)))


### Check: Encoded Value Mappings for Binary Columns

In [38]:
for col, mapping in label_encoders.items():
    print(f"{col} mapping: {{k: int(v) for k, v in mapping.items()}}")

# Save all label encoders


gender mapping: {k: int(v) for k, v in mapping.items()}
partner mapping: {k: int(v) for k, v in mapping.items()}
dependents mapping: {k: int(v) for k, v in mapping.items()}
phoneservice mapping: {k: int(v) for k, v in mapping.items()}
paperlessbilling mapping: {k: int(v) for k, v in mapping.items()}


### Optional: Reverse Mapping for Verification

In [39]:
reverse_map = {v: k for k, v in label_encoders['gender'].items()}
df['gender_decoded'] = df['gender_encoded'].map(reverse_map)
display(df[['gender', 'gender_encoded', 'gender_decoded']].head())


,gender,gender_encoded,gender_decoded
0,Female,0,Female
1,Male,1,Male
2,Male,1,Male
3,Male,1,Male
4,Female,0,Female


### One-Hot Encode Non-Binary Categorical Columns


In [52]:
print(df.columns)


Index(['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'tenure', 'phoneservice', 'multiplelines', 'onlinesecurity',
       'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv',
       'streamingmovies', 'contract', 'paperlessbilling', 'monthlycharges',
       'totalcharges', 'churn', 'contract_encoded', 'gender_encoded',
       'partner_encoded', 'dependents_encoded', 'phoneservice_encoded',
       'paperlessbilling_encoded', 'gender_decoded',
       'internetservice_Fiber optic', 'internetservice_No',
       'paymentmethod_Credit card (automatic)',
       'paymentmethod_Electronic check', 'paymentmethod_Mailed check'],
      dtype='object')


In [51]:
# One-hot encode selected columns
one_hot_cols = ['internetservice', 'paymentmethod']
df = pd.get_dummies(df, columns=one_hot_cols, drop_first=True)


### Check: One-Hot Encoded Columns Contain 0s and 1s

This ensures:

Columns were created (filter)

Encoded values are actually being used (contain both 0s and 1s, or at least some 1s)

In [53]:
for prefix in ['internetservice_', 'paymentmethod_']:
    cols = df.filter(like=prefix).columns
    print(f"\nColumns for '{prefix}': {list(cols)}")
    for col in cols:
        print(f"{col} value counts:\n{df[col].value_counts()}\n")





Columns for 'internetservice_': ['internetservice_Fiber optic', 'internetservice_No']
internetservice_Fiber optic value counts:
internetservice_Fiber optic
False    3936
True     3096
Name: count, dtype: int64

internetservice_No value counts:
internetservice_No
False    5512
True     1520
Name: count, dtype: int64


Columns for 'paymentmethod_': ['paymentmethod_Credit card (automatic)', 'paymentmethod_Electronic check', 'paymentmethod_Mailed check']
paymentmethod_Credit card (automatic) value counts:
paymentmethod_Credit card (automatic)
False    5511
True     1521
Name: count, dtype: int64

paymentmethod_Electronic check value counts:
paymentmethod_Electronic check
False    4667
True     2365
Name: count, dtype: int64

paymentmethod_Mailed check value counts:
paymentmethod_Mailed check
False    5428
True     1604
Name: count, dtype: int64

